### Carga del Dataset

In [8]:
import pandas as pd

ruta_csv = "data\StudentsPerformance.csv"

df = pd.read_csv(ruta_csv)
print("Dimensiones del dataset:", df.shape)
df.head()

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Claudia\AppData\Local\Temp\ipykernel_7980\2894486688.py:3: SyntaxWarning: invalid escape sequence '\S'
  ruta_csv = "data\StudentsPerformance.csv"


Dimensiones del dataset: (1000, 8)


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [9]:
print(df.columns)

columnas_numericas = ['math score', 'reading score', 'writing score']

# Subconjunto de datos solo con esas columnas
X = df[columnas_numericas].copy()

# Comprobación rápida de valores faltantes
print("\nValores nulos por columna:")
print(X.isnull().sum())

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

Valores nulos por columna:
math score       0
reading score    0
writing score    0
dtype: int64
